# アキネイター実装編

これから説明を交えながらアキネイターを実装していきます！  
分からないことがあれば適宜質問するようにして下さい．

# 0.環境準備

## 0-1. モジュール
まずはモジュールというものを色々インストールします．  
これは，ただ実行するだけでOKです．

In [ ]:
from google.colab import auth
import gspread
from google.auth import default
import pandas as pd
import random
# !pip install japanize-matplotlib
import numpy as np
import matplotlib.pyplot as plt
import japanize_matplotlib

## 0-2. 質問関係の設定
ここで，質問内容をまとめたものを変数に入れておきます．  
ここで使われているのは Python の *辞書(dictionary)* という型です．  
辞書についてはパワーポイントを見てください．  
ここは，先生たちがみんなのアンケートを参考に作ったので，そのまま実行してください．  
なにかあれば，伝えてください！

In [ ]:
questions_org = {
  "grade": {
    "choices": ["1年", "2年", "3年"],
    "statement": "{0}生ですか？"
  },
  "class": {
    "choices": ["1組", "2組"],
    "statement": "{0}ですか？"
  },
  "attendance_num": {
    "choices": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35],
    "statement": "出席番号は{0}番ですか？"
  },
  "sex": {
    "choices": ["女", "男"],
    "statement": "{0}性ですか？"
  },
  "club": {
    "choices": ["ESS部", "その他の部", "バスケットボール部", "剣道部", "卓球部", "放送映像文化部", "未所属", "自然科学部", "茶道部", "陸上部", "音楽部"],
    "statement": "{0}ですか？"
  },
  "subject": {
    "choices": ["国語", "数学", "理科", "社会", "英語"],
    "statement": "最も好きな教科は{0}ですか？"
  },
  "door": {
    "choices": ["アウトドア派", "インドア派"],
    "statement": "{0}ですか？"
  },
  "bus": {
    "choices": ["上郡方面", "新宮方面", "相生方面"],
    "statement": "登下校時のバスは{0}ですか？"
  },
  "fishmeet": {
    "choices": ["肉派", "魚派"],
    "statement": "{0}ですか？"
  },
  "blood": {
    "choices": ["朝型", "昼型", "夜型"],
    "statement": "{0}ですか？"
  },
  "kinotake": {
    "choices": ["きのこの山", "たけのこの里"],
    "statement": "{0}派ですか？"
  },
  "ricebread": {
    "choices": ["ご飯", "パン"],
    "statement": "朝食は{0}派ですか？"
  },
  "season": {
    "choices": ["春", "夏", "秋", "冬"],
    "statement": "好きな季節は{0}ですか？"
  },
  "birthday": {
    "choices": ["1月", "2月", "3月", "4月", "5月", "6月", "7月", "8月", "9月", "10月", "11月", "12月"],
    "statement": "{0}生まれですか？"
  },
  "living": {
    "choices": ["たつの市", "上郡町", "太子町", "姫路市", "宍粟市", "明石市", "相生市", "赤穂市"],
    "statement": "{0}に住んでいますか？"
  },
  "income": {
    "choices": ["はい", "いいえ"],
    "statement": "両親は共働きですか？"
  },
  "committee": {
    "choices": ["はい", "いいえ"],
    "statement": "前期に委員会に入っていましたか？"
  },
  "3tennkai": {
    "choices": ["A", "B", "C"],
    "statement": "三展開 (2023.10.21時点)のクラスは{0}ですか？"
  },
  "heyfever": {
    "choices": ["はい", "いいえ"],
    "statement": "花粉症ですか？"
  },
  "grasses": {
    "choices": ["はい", "いいえ"],
    "statement": "メガネをかけていますか？（授業中のみかけている人はいいえに）"
  },
  "science": {
    "choices": ["はい", "いいえ"],
    "statement": "科学は好きですか？"
  },
  "apple": {
    "choices": ["はい", "いいえ"],
    "statement": "iphone,ipadなど、apple製品をもっていますか？"
  },
  "learning": {
    "choices": ["どっちも好き", "どっちも嫌い", "勉強", "運動"],
    "statement": "運動と勉強だと{0}の方が好きですか？"
  },
  "asmr": {
    "choices": ["はい", "いいえ"],
    "statement": "ASMRは好きですか？"
  },
  "food": {
    "choices": ["はい", "いいえ"],
    "statement": "食べ物の好き嫌いはありますか"
  }
}

## 0-3. スプレッドシートの設定
まず，皆のドライブの中にある「Python班(回答).xlsx」を開いてください．  
開いたら，左上の「ファイル」タブを押してください．  
押したら，「Googleスプレッドシートとして保存」を押したら完了です．  
  
次に，下のセルを変更せずに実行してください．  
実行すると，ウィンドウが出ると思います．  
そうしたら，自分のアカウントを選んでログインし，次の画面は下の方に行って  
これで完了です．

In [ ]:
# googleスプレッドシートの設定
# !!! 変更しないで実行してください !!!
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

## 0-4. スプレッドシートを DataFrame に変更
少し難しいので，ここは実行するだけにしてください！

In [ ]:
# googleスプレッドシートからDataFrameに変換
# !!! 変更しないで実行してください !!!
url = "https://docs.google.com/spreadsheets/d/1sYQtBwc_CIaa3BeSU8Ga-9q8iSom9TfVk8t9ITIop14"
ss = gc.open_by_url(url)
st = ss.get_worksheet(0)
df_org = pd.DataFrame.from_records(st.get_all_values())
df_org

## 0-5. DataFrame の加工
スプレッドシートなどの表のデータから DataFrame に変換した後は，不要なデータがあったり，扱いにくいままだったりすることが大半です．  
こういった際は，まず綺麗に加工してあげる必要があります．

### 1. 不要な列の削除
この中でアキネイターを作るにあたって，「タイムスタンプ」，「メールアドレス」，「スコア」は不要です．  
こういったとき，DataFrame（以下:df）では列（横）を切り取ることができます．

dfの列を a 番目から b 番目まで切り取りたいときは，  

`df.iloc[:, a : b + 1]`  ※  b + 1 の部分は自分で計算して入れてください．例えば 6番目 までのときは， 7 を入力してください．  

という風に書きます．実際にやってみよう！  

`df_org`のdfを使って，`df_1` の変数に入れよう．

In [ ]:
# ここから　　　※今の df の名前は df_org なことに注意！
df_1 = df_org.iloc[:, 2:28]
# ここまで
display(df_1)

| |2|3|4|5|6|7|8|9|10|11|...|18|19|20|21|22|23|24|25|26|27|
|:----|:----|:----|:----|:----|:----|:----|:----|:----|:----|:----|:----|:----|:----|:----|:----|:----|:----|:----|:----|:----|:----|
|0|名前を..|学年..|クラス..|出席番号...|性別...|部活...|教科...|どちら...|バス..|どちら...|...|両親は...|委員会...|三展開...|花粉症...|メガネ...|科学...|apple...|運動...|ASMR...|食べ物...|
|1|安達 友香|1年|1組|1|女|バスケットボール部|英語|インドア派|相生方面|魚派|...|いいえ|いいえ|A|はい|はい|はい|はい|運動|はい|はい|
|2|天川　颯|1年|1組|2|男|自然科学部|英語|インドア派|相生方面|魚派|...|いいえ|はい|C|はい|はい|はい|はい|どっちも好き|はい|はい|
|3|有井　万修|1年|1組|3|男|自然科学部|理科|インドア派|相生方面|肉派|...|はい|いいえ|A|はい|いいえ|はい|いいえ|運動|いいえ|はい|
|4|池本ひなた|1年|1組|4|女|放送映像文化部|数学|インドア派|相生方面|魚派|...|はい|いいえ|C|はい|はい|はい|はい|どっちも嫌い|いいえ|いいえ|

以上のように上の番号が２～２７まであればOKです．

---

### 2. 不要な行を削除
データにおいて，0行目の質問の言葉たちが要りません．

dfの行（縦）を a 番目から b 番目まで切り取りたいときは，

`df.iloc[a : b + 1]` ※ b + 1 の部分は自分で計算して入れてください．例えば 6番目 までのときは， 7 を入力してください．

という風に書きます．実際にやってみよう！（少しひっかけ要素あります）

`df_1` のdfを使って，`df_2`  の変数に入れよう．

In [ ]:
# ここから　　　※今の df の名前は df_org なことに注意！
df_2 = df_1.iloc[1:]
# ここまで
display(df_2)

|2|3|4|5|6|7|8|9|10|11|...|18|19|20|21|22|23|24|25|26|27|
|:----|:----|:----|:----|:----|:----|:----|:----|:----|:----|:----|:----|:----|:----|:----|:----|:----|:----|:----|:----|:----|
|1|安達 友香|1年|1組|1|女|バスケットボール部|英語|インドア派|相生方面|魚派|...|いいえ|いいえ|A|はい|はい|はい|はい|運動|はい|はい|
|2|天川　颯|1年|1組|2|男|自然科学部|英語|インドア派|相生方面|魚派|...|いいえ|はい|C|はい|はい|はい|はい|どっちも好き|はい|はい|
|3|有井　万修|1年|1組|3|男|自然科学部|理科|インドア派|相生方面|肉派|...|はい|いいえ|A|はい|いいえ|はい|いいえ|運動|いいえ|はい|
|4|池本ひなた|1年|1組|4|女|放送映像文化部|数学|インドア派|相生方面|魚派|...|はい|いいえ|C|はい|はい|はい|はい|どっちも嫌い|いいえ|いいえ|

以上のようになればOKです．

---

### 3. 行の名前を変更
今，列の名前は上の番号の２～２７になっていますが，列の名前としては，各質問の英語の名前を入れたいです．

各質問の英語は最初の方で作った，質問の辞書を使います．  
各質問の英語は，質問の辞書のキーの部分だけを使うので， `questions_org.keys()` で，取り出せます．  
今回は，それに加えて名前が先頭につくので，`"name"` も先頭に入れてあげないといけません．  

なので，加えたものを変数`key`として作るので，それを皆は使って下さい！

列の名前を変更するときは， `df.columns = 〇〇` で変更できます．

それではやってみよう．

`df_3` を加工しよう．

In [ ]:
df_3 = df_2.copy()
key = ["name"] + list(questions_org.keys())
# ここから　　　※今の df の名前は df_1 なことに注意！
df_3.columns = key
# ここまで
display(df_3)

|name|grade|class|attendance_num|sex|club|subject|door|bus|fishmeet|...|income|committee|3tennkai|heyfever|grasses|science|apple|learning|asmr|food|
|:----|:----|:----|:----|:----|:----|:----|:----|:----|:----|:----|:----|:----|:----|:----|:----|:----|:----|:----|:----|:----|
|1|安達 友香|1年|1組|1|女|バスケットボール部|英語|インドア派|相生方面|魚派|...|いいえ|いいえ|A|はい|はい|はい|はい|運動|はい|はい|
|2|天川　颯|1年|1組|2|男|自然科学部|英語|インドア派|相生方面|魚派|...|いいえ|はい|C|はい|はい|はい|はい|どっちも好き|はい|はい|
|3|有井　万修|1年|1組|3|男|自然科学部|理科|インドア派|相生方面|肉派|...|はい|いいえ|A|はい|いいえ|はい|いいえ|運動|いいえ|はい|
|4|池本ひなた|1年|1組|4|女|放送映像文化部|数学|インドア派|相生方面|魚派|...|はい|いいえ|C|はい|はい|はい|はい|どっちも嫌い|いいえ|いいえ|

以上のようになればOKです．

加工は終了です！きれいになりました．

---

# １．ゲームのパーツを作る
さて，ここからが本題です．ゲームの中のパーツについて理解しましょう．

## 1-1. 質問を選ぶ
まず，数ある質問の中から質問を選ばなくてはなりません．  
選び方は，ランダムではありますが，ある程度考える必要があります．  
ここは，皆の意見を求めますが，とりあえず先生が考えてきたことで進めます．

###1. 重要度別に分ける

質問には重要度があるはずです．
2択の質問は，2択しかないので，人物を絞りにくいです．つまり重要度が低いです．  
対して，選択肢が多ければ，人物ウを絞りやすいので，重要度は高いと言えます．  
また，出席番号や，学年などは決められた数しかないので，重要度は高いと考えます．

以上のことを踏まえて，先生は以下のように分けました．  
ーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーー

特に重要（essential）：

出席番号，部活，誕生日，住所，

重要(important)：

学年，クラス，教科，バス，四季，三展開，運動勉強

そこまで(normal)：

性別，アウトドア派，肉魚，朝昼夜，きのこたけのこ，ご飯パン，両親，委員会，花粉症，眼鏡，科学，apple，ASMR，食べ物

ーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーー

これを設定します．
これは，辞書とリストを使って，設定します．

normalは設定してあるので，参考にしながら，他のところも設定してみてください．

入れる値は，各質問の英語を参照してください．

In [ ]:
important_org = {
    "essential":[
        # ここから

        # ここまで
        ],
    "important":[
        # ここから

        # ここまで
        ],
    "normal":[
        "sex","door","fishmeet", "blood","kinotake", "ricebread", "income","committee","heyfever", "grasses", "science", "apple", "asmr", "food"
        ]
}

In [ ]:
important_org = {
    "essential":["attendance_num", "club", "birthday", "living"],
    "important":["grade", "class", "subject", "bus", "season", "3tennkai", "learning"],
    "normal":["sex","door","fishmeet", "blood","kinotake", "ricebread", "income","committee",
              "heyfever", "grasses", "science", "apple", "asmr", "food"]
}

###2. 残っている生徒数で重要度を選択する
生徒数が減ってきたら normal -> important -> essential という風に重要度を挙げていきたいです．

前習った「もし～なら」を使います．

```python
if 〇〇:     # もし〇〇なら

elif △△:   # 〇〇ではなくて，△△なら

else:  # それ以外なら
```

という感じでしたよね．

もし，残っている生徒数が， 90人より大きいなら 変数`importance`に`"normal"`を，  
6人より大きいなら変数`importance`に`"important"`を，
そうでなければ，変数`importance`に`"essential"`を入れよう．

残っている生徒数は `len(df)`で使えます．

In [ ]:
df = df_3.copy()
# ここから

# ここまで

In [ ]:
if len(df) > 90:
  importance = "normal"
elif len(df) > 6:
  importance = "important"
else:
  importance = "essential"

これで準備完了！選びます．  
以下を実行してください．今は"normal"の中から選ばれます．

In [ ]:
# 質問の選択
content = random.choice(questions_dic[importance]) # 質問の種類
statement = question_data[content]["statement"] # 質問文
choices = list(set(df[content])) # 質問の選択肢
q_target = random.choice(choices) # 回答対象

print(content)
print(statement.format(q_target))

##1-2. 生徒の絞り込み
質問の答え方によって，生徒を絞り込みます．  
生徒の情報はdfに入っているので，dfを絞ることになります．
ひとまず，dfを絞り込んでみましょう．

「好きな教科は数学である」生徒を絞り込んでみましょう．
この時は，
```python
df[df["subject"] == "数学"]
```
という風に書くと絞り込めます．実行して見ましょう．

In [ ]:
df[df["subject"] == "数学"]

"subject" の部分を見ればわかる通り，「数学」の人だけ絞り込めました．  
色々やってみましょう．

In [ ]:
# 1. 「部活がバスケットボール部である」生徒を絞り込みましょう



In [ ]:
# 2. 「眼鏡をかけている」生徒を絞り込みましょう



In [ ]:
# 3. 自由にどうぞ



掴めましたかね？  
ここからは色々分析して見ましょう．

#2. 分析
集まったデータについて分析してみましょう．
今回は先生が考えた分析をいくつか紹介します．

分かったことは，ポスターに書いてもいいと思います．

##2-1. 解答の棒グラフ化
質問ごとに，各回答がどれだけ集まったのか見てわかるように棒グラフで表します．

In [ ]:
content = "subject"

labels = np.array(questions_org[content]["choices"])
amounts = []
for label in labels:
  data = len(df[df[content]==label])
  amounts.append(data)
amounts = np.array(amounts)
print(amounts)
plt.bar(labels, amounts)
plt.title(content)

##2-2. 散布図
二つの質問の回答の関係性を表します．

In [ ]:
content_1 = "subject"
answer_1 = "数学"

content_2 = "science"
answer_2 = "はい"



In [ ]:

# 初期化

important_org = {
    "essential":["attendance_num", "club", "birthday", "living"],
    "important":["grade", "class", "subject", "bus", "season", "3tennkai", "learning"],
    "normal":["sex","door","fishmeet", "blood","kinotake", "ricebread", "income","committee",
              "heyfever", "grasses", "science", "apple", "asmr", "food"]
}
question_data = questions_org.copy()
df = df_org.copy()
questions_dic = important_org.copy()
yes_no = {0: "はい", 1 : 'いいえ'}
# 質問の回数を保存
N = 0

while True:
    # 生徒数が1以下なら終わり
    if len(df) < 1:
        print("対象者が見つかりませんでした")
        # display(df_pre)
        # print(f"\n第{N}問")
        # print("質問 : ", statement.format(q_target))
        # print("答え : ", yes_no[ans])
        break
    elif len(df) == 1:
        print(df["name"].to_list()[0])
        break

    # 質問内容を選ぶ

    # 質問の重要性を考える
    if len(df) > 90:
      importance = "normal"
    elif len(df) > 6:
      importance = "important"
    else:
      importance = "essential"

    # 重要性別のリストが空なら多い normal から選ぶ
    if len(questions_dic[importance]) < 1:
      importance = "normal"

    # 質問の選択
    content = random.choice(questions_dic[importance])

    questions_dic[importance].remove(content) # 削除
    statement = question_data[content]["statement"] # 質問文

    # 質問の選択肢
    choices = list(set(df[content]))

    # 選択肢が1つの場合飛ばす
    if len(choices) < 2:
      continue

    # 答える
    q_target = random.choice(choices) # 回答対象

    ans = random.randint(0, 1) # はい / いいえ


    N += 1
    print(f"\n第{N}問")
    print("質問 : ", statement.format(q_target))
    print("答え : ", yes_no[ans])

    df_pre = df.copy()

    if ans == 0: # Yes
        df = df[df[content] == q_target]
    else: # No
        df = df[df[content] != q_target]

    # 聞いた質問を省く
    # del question_data[content]

    # display(df)
    # break
# display(df)


In [ ]:
M = 1
for _ in range(M):
  func()